In [ ]:
# script to add training reactions to a training reactions reaction.py and dictionary.txt

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import rmgpy.data.thermo
import rmgpy.data.kinetics
import rmgpy.reaction
import rmgpy.species
import rmgpy

import rmgpy.chemkin

In [2]:
def duplicate_exists(test_entry):
    for entry in training_depo.entries:
        if training_depo.entries[entry].item.is_isomorphic(test_entry.item):
            return True
    return False

In [3]:
# load the new training reactions
new_training_rxns = 'new_kinetics'  # path to training reactions
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(new_training_rxns)
print(f'{len(ark_kinetics_database.libraries[""].entries)} new reactions loaded')

48 new reactions loaded


In [4]:
# Load Kinetics Database (quick, automatically loads recommended families)
family = 'Disproportionation'
ref_library_path = os.path.join(rmgpy.settings['database.directory'], 'kinetics')
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[],
    families=[family]
)
training_depo = ref_database.families[family].get_training_depository()
print(len(training_depo.entries), 'training reactions')

137 training reactions


In [5]:
species_dict = training_depo.get_species(os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training', 'dictionary.txt'))

In [6]:
def get_species(sp):
    # look for the species in the training depo and return it if it already exists
    for key in species_dict.keys():
        if sp.is_isomorphic(species_dict[key]):
            return species_dict[key]

In [8]:
# go through the new training reactions and make sure they use existing species dictionary definition
for entry in ark_kinetics_database.libraries[""].entries:
    for i in range(0, len(ark_kinetics_database.libraries[""].entries[entry].item.reactants)):
        official_sp = get_species(ark_kinetics_database.libraries[""].entries[entry].item.reactants[i])
        if official_sp:
            ark_kinetics_database.libraries[""].entries[entry].item.reactants[i] = official_sp

    for i in range(0, len(ark_kinetics_database.libraries[""].entries[entry].item.products)):
        official_sp = get_species(ark_kinetics_database.libraries[""].entries[entry].item.products[i])
        if official_sp:
            ark_kinetics_database.libraries[""].entries[entry].item.products[i] = official_sp


In [9]:
# Splice the new training reactions into the training depo
index_start = 100

for i, entry in enumerate(ark_kinetics_database.libraries[''].entries):
    
    if duplicate_exists(ark_kinetics_database.libraries[''].entries[entry]):
        continue
    if ark_kinetics_database.libraries[''].entries[entry].item.elementary_high_p:
        # manual fix to this issue: https://github.com/ReactionMechanismGenerator/RMG-Py/issues/2341
        ark_kinetics_database.libraries[''].entries[entry].item.elementary_high_p = False
    
    j = index_start + i
    while j in training_depo.entries.keys():
        j += 1
        continue
#     print(j, ark_kinetics_database.libraries[''].entries[entry])
    training_depo.entries[j] = ark_kinetics_database.libraries[''].entries[entry]
print(len(training_depo.entries), 'training reactions')

185 training reactions


In [10]:
# Save the results somewhere else
training_dir = os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training')
training_depo.save_dictionary(os.path.join(training_dir, 'dictionary.txt'))
training_depo.save(os.path.join(training_dir, 'reactions.py'))

CH3CO(20) + HO2(16) <=> H2O2(17) + CH2CO(24)
CH2CHO(21) + HO2(16) <=> H2O2(17) + CH2CO(24)
SC4H9(183) + HO2(16) <=> H2O2(17) + C4H8(189)
SC4H9(183) + HO2(16) <=> H2O2(17) + C4H8(188)
SC4H9(183) + HCO(19) <=> CH2O(9) + C4H8(188)
SC4H9(183) + CH3CO(20) <=> CH3CHO(35) + C4H8(188)
CH3O(31) + C3H5-A(94) <=> C3H6(12) + CH2O(9)
SC4H9(183) + C3H5-A(94) <=> C3H6(12) + C4H8(188)
C3H5-A(94) + CH2CHO(21) <=> C3H6(12) + CH2CO(24)
IC3H7(93) + C4H8(748) <=> SC4H9(183) + C3H6(12)
C4H7(190) + CH2CHO(21) <=> C4H8(189) + CH2CO(24)
C4H7(190) + CH3O(31) <=> C4H8(189) + CH2O(9)
C4H7(190) + SC4H9(183) <=> C4H8(188) + C4H8(189)
C4H7(190) + SC4H9(183) <=> C4H8(188) + C4H8(188)
C4H7(190) + HO2(16) <=> H2O2(17) + C4H6(194)
C4H7(190) + CH3(18) <=> CH4(10) + C4H6(194)
C4H7(190) + C4H7(190) <=> C4H8(189) + C4H6(194)
C4H7(190) + SC4H9(183) <=> butane(1) + C4H6(194)
C4H7(190) + CH2CHO(21) <=> CH3CHO(35) + C4H6(194)
C4H7(190) + CH2(23) <=> CH3(18) + C4H6(194)
SC4H9(183) + C4H7(191) <=> C4H8(188) + C4H8(188)
SC4H9(183)